In [1]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


# 下載檔案

In [ ]:
#解壓縮檔案
#%cp -r "/content/MyDrive/MyDrive/UT_NLP/dataset/drive_data.zip" /content/sample_data/drive_data.zip

In [ ]:
#解壓縮檔案
#!unzip /content/sample_data/drive_data.zip -d /content/a4-distrib

In [2]:
#複製檔案
!cp -r /content/MyDrive/MyDrive/UT_NLP/final/ /content/

In [ ]:
#解壓縮檔案
#!unzip /content/sample_data/dense_data.zip -d /content/a4-distrib

In [3]:
%cd /content/final/

/content/final


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00


In [5]:
!pip install checklist
!jupyter nbextension install --py --sys-prefix checklist.viewer
!jupyter nbextension enable --py --sys-prefix checklist.viewer
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 105.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 36.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached qtconsole-5.5.1-py3-none-any.whl (123 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.4/348.4 kB 35.0 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using 

# Load DataSet

In [6]:
from transformers import pipeline

model = pipeline('question-answering', model="distilbert-base-cased-distilled-squad")
%load_ext autoreload
%autoreload 2

import checklist
import itertools

import checklist.editor
import checklist.text_generation
from checklist.expect import Expect
import numpy as np
from checklist.perturb import Perturb
import datasets
import pandas as pd
import json

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [51]:
# SAVE SQUAD DATASET TO CSV

dataset_qa = datasets.load_dataset('squad')
adver_qa_train = datasets.load_dataset('adversarial_qa', 'adversarialQA')
adver_qa_train = adver_qa_train.remove_columns("metadata")
def format_dataset(example):
    """
    format answers from dict to json
    so that data looks consistent when exporting to csv
    """
    example['answers'] = json.dumps(example['answers'])
    return example

dataset = datasets.concatenate_datasets([dataset_qa['train'],adver_qa_train['train']])#
dataset = dataset.map(format_dataset)
dataset.to_csv('train_merge.csv', index=None)
df_train = pd.read_csv('train_merge.csv')
df_train = pd.read_csv('train_merge.csv')
df_train_selected = df_train[['context', 'question', 'answers']]
df_train_selected

Creating CSV from Arrow format:   0%|          | 0/118 [00:00<?, ?ba/s]

,context,question,answers
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{""text"": [""Saint Bernadette Soubirous""], ""answ..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{""text"": [""a copper statue of Christ""], ""answe..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{""text"": [""the Main Building""], ""answer_start""..."
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,"{""text"": [""a Marian place of prayer and reflec..."
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,"{""text"": [""a golden statue of the Virgin Mary""..."
...,...,...,...
117594,"Some high-speed black-and-white films, such as...",What is the lowest ISO mentioned?,"{""text"": [""400""], ""answer_start"": [543]}"
117595,"Some high-speed black-and-white films, such as...",What is the highest ISO mentioned?,"{""text"": [""3200""], ""answer_start"": [60]}"
117596,"Some high-speed black-and-white films, such as...",What is Kodak'sbrand name of E6 film?,"{""text"": [""Ektachrome""], ""answer_start"": [470]}"
117597,"Some high-speed black-and-white films, such as...",How do these films differ?,"{""text"": [""high-speed black-and-white films""],..."


In [19]:
import pandas as pd
def dataset_fmt(t, file_name):
    """
    format t to acceptable dataframe format
    """
    df = pd.DataFrame([
        {'context':i[k][0], 'question':i[k][1], 'answers_text': j[k]}
        for i, j in zip(t.data, t.labels)
        for k in range(len(i))
    ])
    df['answer_start'] = df.apply(lambda row: row['context'].index(row['answers_text']), axis=1)
    df['answers'] = df.apply(lambda row: {'text': [row['answers_text']], 'answer_start': [row['answer_start']]}, axis=1)
    df = df.drop(columns=['answers_text', 'answer_start'])
    df['answers'] = df['answers'].apply(json.dumps)
    df = df.drop_duplicates().reset_index(drop=True)
    df.to_csv(f'new_data/{file_name}.csv', index=False)
    return df


def inv_dataset_fmt(t, file_name, df_train=df_train):
    """
    use original answers for invariance cases
    note that in many cases start position changes, that's why we find the correct index
    """
    df = pd.DataFrame([{'context':i[0][0], 'question':i[0][1], 'context_modified':i[1][0], 'question_modified':i[1][1]} for i in t.data])
    df = (
        df
        .merge(df_train, on=['context','question'], how='inner')
        .drop(columns=['context', 'question', 'id', 'title'])
        .rename(columns={'context_modified': 'context', 'question_modified':'question'})
        .drop_duplicates(subset=['context','question'])
        .reset_index(drop=True)
    )
    # df = find_matching_index(df)
    df['answers'] = df['answers'].apply(json.dumps)
    df.to_csv(f'new_data/{file_name}.csv', index=False)
    return df

In [20]:
import re
def find_all_matching(string, sentence):
    matching_lst = []
    for m in re.finditer(re.escape(string), sentence): #re.escape deals with weird character issues
        matching_lst.append(m.start())
    return matching_lst

def find_matching_index(df):
    for index, row in df.iterrows():
        row['answers'] = json.loads(row['answers'])
        for i in range(len(row['answers']['text'])):
            original_index = row['answers']['answer_start'][i]
            all_matched_indexes = find_all_matching(row['answers']['text'][i], row['context'])
            if len(all_matched_indexes) ==0:
                pass
            elif original_index in all_matched_indexes:
                # if the original index is in the matched indexes, do nothing
                pass
            else:
                print(index)
                print(row)
                print('changed')
                # if not, choose the index that's the closest to the original index
                row['answers']['answer_start'][i] = min(all_matched_indexes, key=lambda x: abs(x - original_index))
                print(original_index)
                print(row['answers']['answer_start'][i])
    return df
editor = checklist.editor.Editor()
editor.tg

In [21]:
#1117 version
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser
from transformers import pipeline

model_checkpoint = "trained_model_qa_64/trained_model_qa_64/checkpoint-4000"
baseline_model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
baseline_model_pipeline = pipeline('question-answering', model=baseline_model, tokenizer=tokenizer)

#model = pipeline('question-answering', model="distilbert-base-cased-distilled-squad")
baseline_model_pipeline({
    'context': 'A new strain of flu that has the potential to become a pandemic has been identified by scientists.',
    'question': 'What has been discovered by scientists?'
})

{'score': 0.5901924967765808,
 'start': 0,
 'end': 19,
 'answer': 'A new strain of flu'}

# Checklist test

In [22]:
#1117 version

from checklist.test_suite import TestSuite
def predconfs(context_question_pairs):
    """
    output: predictions, confidence
    source: https://github.com/marcotcr/checklist/blob/115f123de47ab015b2c3a6baebaffb40bab80c9f/notebooks/tutorials/5.%20Testing%20transformer%20pipelines.ipynb
    https://github.com/filipegl/checklist_SQuAD_exp/blob/main/1.%20SQuAD-create-test-suite.ipynb
    https://github.com/marcotcr/checklist/blob/4e6e5e33a26f30c20ed602b2050f6c73e123cc23/notebooks/Sentiment.ipynb
    https://github.com/marcotcr/checklist/tree/4e6e5e33a26f30c20ed602b2050f6c73e123cc23
    """
    preds = []
    confs = []
    for c, q in context_question_pairs:
        try:
            p = baseline_model_pipeline(question=q, context=c, truncation=True, )
        except:
            print('Failed', q)
            preds.append(' ')
            confs.append(1)
        preds.append(p['answer'])
        confs.append(p['score'])
    return preds, np.array(confs)

def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

def format_squad(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'Q: %s\n' % (q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret
editor = checklist.editor.Editor()
suite = TestSuite()

In [23]:
#1117 Vocabulary
suggestions1 = editor.suggest('{first_name} is {mask} than {first_name2}.')[:60]
print(suggestions1)
suggestions1 = [x for x in suggestions1 if x.endswith("er")][:20]

['smarter', 'older', 'better', 'younger', 'taller', 'worse', 'stronger', 'different', 'shorter', 'cooler', 'nicer', 'tougher', 'bigger', 'hotter', 'happier', 'smaller', 'wiser', 'more', 'faster', 'richer', 'darker', 'thinner', 'weaker', 'less', 'larger', 'quieter', 'cleaner', 'heavier', 'healthier', 'closer', 'colder', 'slower', 'wealthier', 'quicker', 'longer', 'harder', 'safer', 'lighter', 'warmer', 'brighter', 'cheaper', 'sharper', 'higher', 'louder', 'thicker', 'greater', 'lower', 'easier', 'deeper', 'poorer', 'softer', 'smoother', 'simpler', 'stranger', 'newer', 'other', 'superior', 'clearer', 'stricter', 'tighter']


# Vocabulary

In [24]:
adj = ['old', 'smart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]
t = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is {adj[0]}er?'
    )
    ],
    labels = ['{first_name}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=500,
    save=True
    )
name = 'A is COMP than B. Who is more COMP?'
# description = ''
# test = MFT(**t, name=name, description=description, capability='Vocabulary')
# suite.add(test)
dataset_fmt(t, "compare_more")

,context,question,answers
0,Jim is darker than Annie.,Who is darker?,"{""text"": [""Jim""], ""answer_start"": [0]}"
1,Anthony is taller than Jean.,Who is taller?,"{""text"": [""Anthony""], ""answer_start"": [0]}"
2,Donald is older than Laura.,Who is older?,"{""text"": [""Donald""], ""answer_start"": [0]}"
3,Francis is greater than James.,Who is greater?,"{""text"": [""Francis""], ""answer_start"": [0]}"
4,Eleanor is darker than Linda.,Who is darker?,"{""text"": [""Eleanor""], ""answer_start"": [0]}"
...,...,...,...
492,Arthur is shorter than Margaret.,Who is shorter?,"{""text"": [""Arthur""], ""answer_start"": [0]}"
493,Joseph is nicer than Albert.,Who is nicer?,"{""text"": [""Joseph""], ""answer_start"": [0]}"
494,Dick is stronger than Leslie.,Who is stronger?,"{""text"": [""Dick""], ""answer_start"": [0]}"
495,Caroline is faster than Patricia.,Who is faster?,"{""text"": [""Caroline""], ""answer_start"": [0]}"


In [25]:
#1117 Vocabulary
#1117 version more than
editor = checklist.editor.Editor()
suite = TestSuite()
t = editor.template(
    [(
    '{first_name} is {adj} than {first_name1}.',
    'Who is {adj}?'
    )
    ],
    labels = ['{first_name}'],
    adj=suggestions1,
    remove_duplicates=True,
    nsamples=500,
    save=True
    )
name = 'A is richer than B. Who is more rich?'
description = ''
#test = MFT(**t, name=name, description=description, capability='Taxonomy')
#suite.add(test)
dataset_fmt(t, "compare_less")

,context,question,answers
0,Eric is better than Leslie.,Who is better?,"{""text"": [""Eric""], ""answer_start"": [0]}"
1,Lisa is nicer than Harold.,Who is nicer?,"{""text"": [""Lisa""], ""answer_start"": [0]}"
2,Tom is nicer than Laura.,Who is nicer?,"{""text"": [""Tom""], ""answer_start"": [0]}"
3,Harold is faster than Joseph.,Who is faster?,"{""text"": [""Harold""], ""answer_start"": [0]}"
4,Tony is stronger than Elizabeth.,Who is stronger?,"{""text"": [""Tony""], ""answer_start"": [0]}"
...,...,...,...
491,Ben is taller than Sara.,Who is taller?,"{""text"": [""Ben""], ""answer_start"": [0]}"
492,Fred is shorter than Harry.,Who is shorter?,"{""text"": [""Fred""], ""answer_start"": [0]}"
493,Sue is shorter than Patricia.,Who is shorter?,"{""text"": [""Sue""], ""answer_start"": [0]}"
494,Ray is better than Steven.,Who is better?,"{""text"": [""Ray""], ""answer_start"": [0]}"


In [26]:
#1117 Intensifiers
#1117 version
def crossproduct(t):
    # takes the output of editor.template and does the cross product of contexts and qas
    ret = []
    ret_labels = []
    for x in t.data:
        cs = x['contexts']
        qas = x['qas']
        d = list(itertools.product(cs, qas))
        ret.append([(x[0], x[1][0]) for x in d])
        ret_labels.append([x[1][1] for x in d])
    t.data = ret
    t.labels = ret_labels
    return t
state = editor.suggest('John is very {mask} about the project.')[:20]
print(', '.join(editor.suggest('John is {mask} {state} about the project.', state=state)[:30]))
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ),
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ),

        ]

    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?'
desc = ''
# test = MFT(**t, name=name, description=desc, capability='Vocabulary')
# suite.add(test)
dataset_fmt(t, "intensifiers_reducers")

very, pretty, extremely, also, still, quite, more, really, not, clearly, fairly, incredibly, particularly, now, understandably, rather, cautiously, surprisingly, certainly, feeling, so, especially, definitely, generally, most, highly, super, reportedly, being, obviously


,context,question,answers
0,Claire is very skeptical about the project. Di...,Who is most skeptical about the project?,"{""text"": [""Claire""], ""answer_start"": [0]}"
1,Claire is very skeptical about the project. Di...,Who is least skeptical about the project?,"{""text"": [""Diana""], ""answer_start"": [44]}"
2,Diana is skeptical about the project. Claire i...,Who is most skeptical about the project?,"{""text"": [""Claire""], ""answer_start"": [38]}"
3,Diana is skeptical about the project. Claire i...,Who is least skeptical about the project?,"{""text"": [""Diana""], ""answer_start"": [0]}"
4,Claire is skeptical about the project. Diana i...,Who is most skeptical about the project?,"{""text"": [""Claire""], ""answer_start"": [0]}"
...,...,...,...
5971,Pamela is slightly confident about the project...,Who is least confident about the project?,"{""text"": [""Pamela""], ""answer_start"": [0]}"
5972,Emily is quite confident about the project. Pa...,Who is most confident about the project?,"{""text"": [""Emily""], ""answer_start"": [0]}"
5973,Emily is quite confident about the project. Pa...,Who is least confident about the project?,"{""text"": [""Pamela""], ""answer_start"": [44]}"
5974,Pamela is slightly confident about the project...,Who is most confident about the project?,"{""text"": [""Emily""], ""answer_start"": [48]}"


# Taxonomy

In [27]:
#1117 Taxonomy
#1117 version
import munch
order = ['size', 'shape', 'age', 'color']
props = []
properties = {
    'color' : ['red', 'blue','yellow', 'green', 'pink', 'white', 'black', 'orange', 'grey', 'purple', 'brown'],
    'size' : ['big', 'small', 'tiny', 'enormous'],
    'age' : ['old', 'new'],
    'shape' : ['round', 'oval', 'square', 'triangular'],
    'material' : ['iron', 'wooden', 'ceramic', 'glass', 'stone']
}
for i in range(len(order)):
    for j in range(i + 1, len(order)):
        p1, p2 = order[i], order[j]
        for v1, v2 in itertools.product(properties[p1], properties[p2]):
            props.append(munch.Munch({
                'p1': p1,
                'p2': p2,
                'v1': v1,
                'v2': v2,
            }))
print(', '.join(editor.suggest('There is {a:p.v1} {p.v2} {mask} in the room.', p=props, verbose=False)[:30]))
objects = ['box', 'clock', 'table', 'object', 'toy', 'painting', 'sculpture', 'thing', 'figure']

t = crossproduct(editor.template(
    {
        'contexts': [
            'There is {a:p.v1} {p.v2} {obj} in the room.',
            'There is {a:obj} in the room. The {obj} is {p.v1} and {p.v2}.',
        ],
        'qas': [
            (
                'What {p.p1} is the {obj}?',
                '{p.v1}'
            ),
            (
                'What {p.p2} is the {obj}?',
                '{p.v2}'
            ),

        ]
    },
    obj=objects,
    p=props,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'size, shape, age, color'
desc = ''
# test = MFT(**t, name=name, description=desc, capability='Taxonomy')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "size_shape_age_color")

sofa, couch, wall, carpet, chair, table, light, lamp, door, clock, mirror, desk, bed, TV, bar, television, window, box, tree, painting, curtain, fan, fridge, screen, wallpaper, piano, rug, shelf, camera, candle


,context,question,answers
0,There is a tiny old box in the room.,What size is the box?,"{""text"": [""tiny""], ""answer_start"": [11]}"
1,There is a tiny old box in the room.,What age is the box?,"{""text"": [""old""], ""answer_start"": [16]}"
2,There is a box in the room. The box is tiny an...,What size is the box?,"{""text"": [""tiny""], ""answer_start"": [39]}"
3,There is a box in the room. The box is tiny an...,What age is the box?,"{""text"": [""old""], ""answer_start"": [48]}"
4,There is a square green box in the room.,What shape is the box?,"{""text"": [""square""], ""answer_start"": [11]}"
...,...,...,...
1663,There is a clock in the room. The clock is big...,What color is the clock?,"{""text"": [""red""], ""answer_start"": [51]}"
1664,There is an old white thing in the room.,What age is the thing?,"{""text"": [""old""], ""answer_start"": [12]}"
1665,There is an old white thing in the room.,What color is the thing?,"{""text"": [""white""], ""answer_start"": [16]}"
1666,There is a thing in the room. The thing is old...,What age is the thing?,"{""text"": [""old""], ""answer_start"": [43]}"


In [28]:
#1117 professions
#1117 version
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')
print(professions)

def clean(string):
    return string.lstrip('[a,the,an,in,at] ').rstrip('.')
def expect_squad(x, pred, conf, label=None, meta=None):
    return clean(pred) == clean(label)
expect_squad = Expect.single(expect_squad)
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ),
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ),

        ]

    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True,
    ))
name = 'Profession vs nationality'
# test = MFT(**t, name=name, expect=expect_squad, description='',  capability='Taxonomy')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "profession_nationality")

['investigator', 'reporter', 'waitress', 'educator', 'executive', 'attorney', 'interpreter', 'economist', 'agent', 'nurse', 'administrator', 'escort', 'model', 'actor', 'DJ', 'historian', 'secretary', 'activist', 'organizer', 'analyst', 'photographer', 'editor', 'assistant', 'entrepreneur', 'investor', 'journalist', 'engineer', 'architect', 'accountant', 'author', 'intern', 'artist', 'producer', 'actress']


,context,question,answers
0,Anne is an Indonesian DJ.,What is Anne's job?,"{""text"": [""DJ""], ""answer_start"": [22]}"
1,Anne is an Indonesian DJ.,What is Anne's nationality?,"{""text"": [""Indonesian""], ""answer_start"": [11]}"
2,Anne is a DJ. Anne is Indonesian.,What is Anne's job?,"{""text"": [""DJ""], ""answer_start"": [10]}"
3,Anne is a DJ. Anne is Indonesian.,What is Anne's nationality?,"{""text"": [""Indonesian""], ""answer_start"": [22]}"
4,Anne is Indonesian. Anne is a DJ.,What is Anne's job?,"{""text"": [""DJ""], ""answer_start"": [30]}"
...,...,...,...
4985,Tim is Chinese. Tim is a waitress.,What is Tim's nationality?,"{""text"": [""Chinese""], ""answer_start"": [7]}"
4986,Tim is Chinese and a waitress.,What is Tim's job?,"{""text"": [""waitress""], ""answer_start"": [21]}"
4987,Tim is Chinese and a waitress.,What is Tim's nationality?,"{""text"": [""Chinese""], ""answer_start"": [7]}"
4988,Tim is a waitress and Chinese.,What is Tim's job?,"{""text"": [""waitress""], ""answer_start"": [9]}"


In [29]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ),
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ),

        ]

    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Animal vs Vehicle'
# test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test, overwrite=True)
dataset_fmt(t, "animal_vehicle")

,context,question,answers
0,Martha has a dog and a SUV.,What animal does Martha have?,"{""text"": [""dog""], ""answer_start"": [13]}"
1,Martha has a dog and a SUV.,What vehicle does Martha have?,"{""text"": [""SUV""], ""answer_start"": [23]}"
2,Martha has a SUV and a dog.,What animal does Martha have?,"{""text"": [""dog""], ""answer_start"": [23]}"
3,Martha has a SUV and a dog.,What vehicle does Martha have?,"{""text"": [""SUV""], ""answer_start"": [13]}"
4,Stephanie has a snake and a train.,What animal does Stephanie have?,"{""text"": [""snake""], ""answer_start"": [16]}"
...,...,...,...
1963,Emily has a firetruck and a fish.,What vehicle does Emily have?,"{""text"": [""firetruck""], ""answer_start"": [12]}"
1964,Tim has a duck and a train.,What animal does Tim have?,"{""text"": [""duck""], ""answer_start"": [10]}"
1965,Tim has a duck and a train.,What vehicle does Tim have?,"{""text"": [""train""], ""answer_start"": [21]}"
1966,Tim has a train and a duck.,What animal does Tim have?,"{""text"": [""duck""], ""answer_start"": [22]}"


In [30]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} bought {a:animal}. {first_name2} bought {a:vehicle}.',
            '{first_name2} bought {a:vehicle}. {first_name} bought {a:animal}.',
        ],
        'qas': [
            (
                'Who bought an animal?',
                '{first_name}'
            ),
            (
                'Who bought a vehicle?',
                '{first_name2}'
            ),

        ]

    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Animal vs Vehicle v2'
# test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test, overwrite=True)
dataset_fmt(t, "animal_vehicle2")

,context,question,answers
0,Martha bought a lizard. Jason bought a truck.,Who bought an animal?,"{""text"": [""Martha""], ""answer_start"": [0]}"
1,Martha bought a lizard. Jason bought a truck.,Who bought a vehicle?,"{""text"": [""Jason""], ""answer_start"": [24]}"
2,Jason bought a truck. Martha bought a lizard.,Who bought an animal?,"{""text"": [""Martha""], ""answer_start"": [22]}"
3,Jason bought a truck. Martha bought a lizard.,Who bought a vehicle?,"{""text"": [""Jason""], ""answer_start"": [0]}"
4,Thomas bought a cat. Andrea bought a motorcycle.,Who bought an animal?,"{""text"": [""Thomas""], ""answer_start"": [0]}"
...,...,...,...
1979,Tim bought a minivan. Eleanor bought an iguana.,Who bought a vehicle?,"{""text"": [""Tim""], ""answer_start"": [0]}"
1980,Rebecca bought a dog. Ashley bought a firetruck.,Who bought an animal?,"{""text"": [""Rebecca""], ""answer_start"": [0]}"
1981,Rebecca bought a dog. Ashley bought a firetruck.,Who bought a vehicle?,"{""text"": [""Ashley""], ""answer_start"": [22]}"
1982,Ashley bought a firetruck. Rebecca bought a dog.,Who bought an animal?,"{""text"": [""Rebecca""], ""answer_start"": [27]}"


In [31]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'),
            ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'),
            ('humble', 'modest'), ('courageous', 'brave'), ('happy', 'joyful'), ('scared', 'frightened'),
           ]

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[0]}. {first_name2} is very {s2[0]}.',
            '{first_name2} is very {s2[0]}. {first_name} is very {s1[0]}.',
        ],
        'qas': [
            (
                'Who is {s1[1]}?',
                '{first_name}'
            ),
            (
                'Who is {s2[1]}?',
                '{first_name2}'
            ),

        ]

    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=250,
    save=True
   ))
t += crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[1]}. {first_name2} is very {s2[1]}.',
            '{first_name2} is very {s2[1]}. {first_name} is very {s1[1]}.',
        ],
        'qas': [
            (
                'Who is {s1[0]}?',
                '{first_name}'
            ),
            (
                'Who is {s2[0]}?',
                '{first_name2}'
            ),

        ]

    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=250,
    save=True
    ))
name = 'Synonyms'
# test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "synonyms")

,context,question,answers
0,Anne is very spiritual. Ryan is very grateful.,Who is religious?,"{""text"": [""Anne""], ""answer_start"": [0]}"
1,Anne is very spiritual. Ryan is very grateful.,Who is thankful?,"{""text"": [""Ryan""], ""answer_start"": [24]}"
2,Ryan is very grateful. Anne is very spiritual.,Who is religious?,"{""text"": [""Anne""], ""answer_start"": [23]}"
3,Ryan is very grateful. Anne is very spiritual.,Who is thankful?,"{""text"": [""Ryan""], ""answer_start"": [0]}"
4,Eleanor is very vocal. Sue is very organized.,Who is outspoken?,"{""text"": [""Eleanor""], ""answer_start"": [0]}"
...,...,...,...
1831,Jonathan is very outspoken. Christine is very ...,Who is vocal?,"{""text"": [""Jonathan""], ""answer_start"": [0]}"
1832,Colin is very outspoken. Dorothy is very organ...,Who is vocal?,"{""text"": [""Colin""], ""answer_start"": [0]}"
1833,Colin is very outspoken. Dorothy is very organ...,Who is organized?,"{""text"": [""Dorothy""], ""answer_start"": [25]}"
1834,Dorothy is very organised. Colin is very outsp...,Who is vocal?,"{""text"": [""Colin""], ""answer_start"": [27]}"


In [32]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs))#list(set(comp_pairs + [(x[1], x[0]) for x in comp_pairs]))
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {comp[0]} than {first_name1}.',
            '{first_name1} is {comp[1]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is {comp[1]}?',
                '{first_name1}',
            ),
            (
                'Who is {comp[0]}?',
                '{first_name}',
            )

        ]
        ,
    },
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'A is COMP than B. Who is antonym(COMP)? B'
# test = MFT(**t, name=name, description='', capability='Taxonomy')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "compare_antonym")

,context,question,answers
0,Alexandra is bigger than Judy.,Who is smaller?,"{""text"": [""Judy""], ""answer_start"": [25]}"
1,Alexandra is bigger than Judy.,Who is bigger?,"{""text"": [""Alexandra""], ""answer_start"": [0]}"
2,Judy is smaller than Alexandra.,Who is smaller?,"{""text"": [""Judy""], ""answer_start"": [0]}"
3,Judy is smaller than Alexandra.,Who is bigger?,"{""text"": [""Alexandra""], ""answer_start"": [21]}"
4,Simon is better than Peter.,Who is worse?,"{""text"": [""Peter""], ""answer_start"": [21]}"
...,...,...,...
1979,Kelly is poorer than Bobby.,Who is richer?,"{""text"": [""Bobby""], ""answer_start"": [21]}"
1980,Betty is taller than Elaine.,Who is shorter?,"{""text"": [""Elaine""], ""answer_start"": [21]}"
1981,Betty is taller than Elaine.,Who is taller?,"{""text"": [""Betty""], ""answer_start"": [0]}"
1982,Elaine is shorter than Betty.,Who is shorter?,"{""text"": [""Elaine""], ""answer_start"": [0]}"


In [33]:
antonym_adjs = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is more {a[0]} than {first_name1}.',
            '{first_name1} is more {a[1]} than {first_name}.',
            '{first_name} is less {a[1]} than {first_name1}.',
            '{first_name1} is less {a[0]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is more {a[0]}?',
                '{first_name}',
            ),
            (
                'Who is less {a[0]}?',
                '{first_name1}',
            ),
            (
                'Who is more {a[1]}?',
                '{first_name1}',
            ),
            (
                'Who is less {a[1]}?',
                '{first_name}',
            ),
        ]
        ,
    },
    a = antonym_adjs,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.'
# test = MFT(**t, name=name, description='', capability='Taxonomy')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "compare_moreless_antonym")

,context,question,answers
0,George is more fat than Melissa.,Who is more fat?,"{""text"": [""George""], ""answer_start"": [0]}"
1,George is more fat than Melissa.,Who is less fat?,"{""text"": [""Melissa""], ""answer_start"": [24]}"
2,George is more fat than Melissa.,Who is more thin?,"{""text"": [""Melissa""], ""answer_start"": [24]}"
3,George is more fat than Melissa.,Who is less thin?,"{""text"": [""George""], ""answer_start"": [0]}"
4,Melissa is more thin than George.,Who is more fat?,"{""text"": [""George""], ""answer_start"": [26]}"
...,...,...,...
7947,Ralph is less responsible than Adam.,Who is less responsible?,"{""text"": [""Ralph""], ""answer_start"": [0]}"
7948,Adam is less irresponsible than Ralph.,Who is more irresponsible?,"{""text"": [""Ralph""], ""answer_start"": [32]}"
7949,Adam is less irresponsible than Ralph.,Who is less irresponsible?,"{""text"": [""Adam""], ""answer_start"": [0]}"
7950,Adam is less irresponsible than Ralph.,Who is more responsible?,"{""text"": [""Adam""], ""answer_start"": [0]}"


# Robustness

In [34]:
# import pickle
# # IF YOU ALREADY RAN THIS CELL, JUMP TO NEXT
# # source: https://github.com/marcotcr/checklist/blob/115f123de47ab015b2c3a6baebaffb40bab80c9f/notebooks/QQP.ipynb
import spacy
nlp = spacy.load('en_core_web_sm')
all_questions = set() # a set of all questions and context
for x in dataset['train']:
    all_questions.add(x['question'])
    all_questions.add(x['context'])


all_questions = list(all_questions)
parsed_questions = list(nlp.pipe(all_questions)) # this takes a while to run
spacy_map = dict([(x, y) for x, y in zip(all_questions, parsed_questions)])


pickle.dump(spacy_map, open('processed_squad.pkl', 'wb'))

In [35]:
import pickle
dataset = datasets.load_dataset('squad')
pairs = [(x['context'], x['question']) for x in dataset['train']]
spacy_map =  pickle.load(open('/content/final/processed_squad.pkl', 'rb'))
processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in pairs]
# check
print("text:",pairs[0][0])
print("entities:",spacy_map[pairs[0][0]].ents)

def question_typo(x):
    """
    x[0]: context
    x[1]: question
    Perturb.add_typos(x[1]): add a typo to question
    """
    return (x[0], Perturb.add_typos(x[1]))
t = Perturb.perturb(pairs, question_typo, nsamples=500)
# test = INV(**t, name='Question typo', capability='Robustness', description='')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad)
# suite.add(test, overwrite=True)
inv_dataset_fmt(t, 'typo')

text: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
entities: (Catholic, the Main Building's, the Virgin Mary., the Main Building, Venite Ad Me Omnes, the Main Building, the Sacred Heart, Grotto, Marian, Lourdes, France, the Virgin Mary, Saint Bernadette Soubirous, 1858, 3, the Gold Dome, Mary)


,context,question,answers
0,A strong and prominent a cappella tradition wa...,When did a cappell abegin to take hold in the ...,"""{\""text\"": [\""1911\""], \""answer_start\"": [98]}"""
1,"Catalan has an inflectional grammar, with two ...",What is used in Catalan a sword endings that i...,"""{\""text\"": [\""consonants\""], \""answer_start\""..."
2,Hong Kong: The event was held in Hong Kong on...,How many torchbearers participated in the rela...,"""{\""text\"": [\""120\""], \""answer_start\"": [568]}"""
3,The tourist industry is heavily based on the p...,What kind of fisihng tourism occurs on the isl...,"""{\""text\"": [\""sportfishing\""], \""answer_start..."
4,A referee may stop the match when they or offi...,Why couldn't Shawn Michaels defend hismelf?,"""{\""text\"": [\""due to excessive blood loss and..."
...,...,...,...
495,The Late Jurassic spans from 163 million to 14...,What caused the destruction of fern prairies a...,"""{\""text\"": [\""Sea levels rose\""], \""answer_st..."
496,A group of seven companies began the developme...,hWat was the goal for USB's?,"""{\""text\"": [\""to make it fundamentally easier..."
497,"After back-to-back pennants in 1880 and 1881, ...",Who assumed ownership of the Cubs fater Hulber...,"""{\""text\"": [\""Spalding\""], \""answer_start\"": ..."
498,Eritrea can be split into three ecoregions. To...,Where si Emba Soira located?,"""{\""text\"": [\""in the center of Eritrea\""], \""..."


In [36]:
def contractions(x):
    conts = Perturb.contractions(x[1])
    return [(x[0], a) for a in conts]
t = Perturb.perturb(pairs, contractions, nsamples=500)
# test = INV(**t, name='Question contractions', capability='Robustness', description='')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad)
# suite.add(test)
inv_dataset_fmt(t, 'contractions')

,context,question,answers
0,Population testing is still being done. Some N...,Who says genetic tests cannot tell the differe...,"""{\""text\"": [\""Geneticists\""], \""answer_start\..."
1,The government of the City of Miami (proper) u...,What's the street address of Miami City Hall?,"""{\""text\"": [\""3500 Pan American Drive\""], \""a..."
2,"In 2010, there were concerns among Tajik offic...",When did Russia say they'll be sending more tr...,"""{\""text\"": [\""2015\""], \""answer_start\"": [644]}"""
3,"Researchers include biologists, geologists, oc...",Who's interested in the effects of harsh envir...,"""{\""text\"": [\""Biologists\""], \""answer_start\""..."
4,Nonverbal communication describes the process ...,What's one example of nonverbal communication?,"""{\""text\"": [\""body language\""], \""answer_star..."
...,...,...,...
495,The Bamar form an estimated 68% of the populat...,What's the largest percentage of the Burmese ...,"""{\""text\"": [\""Bamar form an estimated 68% of ..."
496,The new SuperSpeed bus provides a fourth trans...,What's full-duplex in SuperSpeed transfer mode?,"""{\""text\"": [\""Communication\""], \""answer_star..."
497,"Israel has two official languages, Hebrew and ...",What're the two official languages of Israel?,"""{\""text\"": [\""Hebrew and Arabic\""], \""answer_..."
498,Einstein partially advocates Mach's principle ...,What's considered contrary to idealist beliefs...,"""{\""text\"": [\""objects exist independently of ..."


In [37]:
#Add random sentence

random_sentences = set()
for x, _ in processed_pairs:
    for y in x.sents:
        random_sentences.add(y.text)
random_sentences = list(random_sentences)
# check
for y in processed_pairs[0][0].sents:
    print(y.text)

def add_random_sentence(x, **kwargs):
    random_s = np.random.choice(random_sentences)
    while random_s in x[0]:
        random_s = np.random.choice(random_sentences)
    random_s = random_s.strip('.') + '. '
    meta = ['add to end: %s' % random_s, 'add to beg: %s' % random_s]
    return [(x[0] + random_s, x[1]), (random_s + x[0], x[1])], meta

def format_add(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s\n' % meta
    return ret

t = Perturb.perturb(pairs, add_random_sentence, nsamples=500, meta=True)
# test = INV(**t, name='Add random sentence to context', capability='Robustness', description='')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_add)
# suite.add(test)
inv_dataset_fmt(t, 'random_sentence')

Architecturally, the school has a Catholic character.
Atop the Main Building's gold dome is a golden statue of the Virgin Mary.
Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".
Next to the Main Building is the Basilica of the Sacred Heart.
Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.
It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.
At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


,context,question,answers
0,"As for Mac OS, System 7 was a 32-bit rewrite f...",How did the Mac System 7 improve multitasking?,"""{\""text\"": [\""co-operative multitasking\""], \..."
1,"During the Hellenistic period, Judea became a ...",What religion rose in Judea durring the Hellen...,"""{\""text\"": [\""Judaism\""], \""answer_start\"": [..."
2,"Historically, the cuisine of Estonia has been ...",What food gathering behaviors are now seen as ...,"""{\""text\"": [\""Hunting and fishing\""], \""answe..."
3,Ptolemy's Geography divided Asia on a similar ...,"""India on this side of the Ganges"" is located ...","""{\""text\"": [\""To the south\""], \""answer_start..."
4,Energy transformations in the universe over ti...,What is a process ultimately using the gravita...,"""{\""text\"": [\""nucleosynthesis\""], \""answer_st..."
...,...,...,...
495,The method of execution of federal prisoners f...,According to what law are federal prisoners ex...,"""{\""text\"": [\""Violent Crime Control and Law E..."
496,Rapid environmental changes typically cause ma...,How many species currently live on earth?,"""{\""text\"": [\""Earth's current species range f..."
497,Religious beliefs in the Eastern Empire and Pe...,In what yer did Muhammad die?,"""{\""text\"": [\""632\""], \""answer_start\"": [468]}"""
498,The annual Southampton Boat Show is held in Se...,What Southampton festival culminates in the Bo...,"""{\""text\"": [\""Sea City\""], \""answer_start\"": ..."


# NER

In [38]:
import re
def change_thing(change_fn):
    def change_both(cq, **kwargs):
        context, question = cq
        a = change_fn(context, meta=True)
        if not a:
            return None
        changed, meta = a
        ret = []
        for c, m in zip(changed, meta):
            new_q = re.sub(r'\b%s\b' % re.escape(m[0]), m[1], question.text)
            ret.append((c, new_q))
        return ret, meta
    return change_both

def expect_same(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    if not meta:
        return pred == orig_pred
    return pred == re.sub(r'\b%s\b' % re.escape(meta[0]), meta[1], orig_pred)

def format_replace(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

def format_replace_context(x, pred, conf, label=None, meta=None):
    ret = format_squad_with_context(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

In [39]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_names), nsamples=500, meta=True)

# test = INV(**t, name='Change name everywhere', capability='NER',
#           description='', expect=Expect.pairwise(expect_same))
# test.run(predconfs, n=100)
# test.summary(3, format_example_fn=format_replace)
# suite.add(test, overwrite=True)
inv_dataset_fmt(t, 'name_change')

,context,question,answers
0,"After just 100 hours of ground combat, and wit...",Why did Coalition nations fear the removal of ...,"""{\""text\"": [\""it would create a power vacuum ..."
1,"Mithridates the Great was the ruler of Pontus,...",How many Romans lived in Mithridate the Great'...,"""{\""text\"": [\""80,000\""], \""answer_start\"": [3..."
2,"On 6 September 2007, Belgian-based Internation...",How much did the Princess Elizabeth station cost?,"""{\""text\"": [\""$16.3 million\""], \""answer_star..."
3,"On September 13, 2009, during the 2009 MTV Vid...",What artist's award reception did Kanye interr...,"""{\""text\"": [\""Taylor Swift\""], \""answer_start..."
4,Eisenhower returned to the U.S. in December 19...,What event contributed to Eisenhower receiving...,"""{\""text\"": [\""Louisiana Maneuvers\""], \""answe..."
...,...,...,...
495,An alternative view offered by Michael Sanders...,To what part of the prey does Michael Sanders ...,"""{\""text\"": [\""the body\""], \""answer_start\"": ..."
496,Beyoncé's vocal range spans four octaves. Jody...,New York Times' Jon Pareles calls Beyoncé's vo...,"""{\""text\"": [\""tart\""], \""answer_start\"": [630]}"""
497,"In 1867, the university opened the first priva...",When did Washington University establish its m...,"""{\""text\"": [\""1891\""], \""answer_start\"": [290]}"""
498,"The team worked on a Wii control scheme, adapt...",What kind of movement interfaced with the swor...,"""{\""text\"": [\""swinging gesture\""], \""answer_s..."


In [40]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_location), nsamples=500, meta=True)

# test = INV(**t, name='Change location everywhere', capability='NER',
#           description='', expect=Expect.pairwise(expect_same))
# test.run(predconfs, n=100)
# test.summary(3, format_example_fn=format_replace)
# suite.add(test, overwrite=True)
inv_dataset_fmt(t, 'location_change')

,context,question,answers
0,Argentine activists told a news conference tha...,What is the name of the activist who promised ...,"""{\""text\"": [\""Jorge Carcavallo\""], \""answer_s..."
1,"By 1976, Queen were back in the studio recordi...",What Queen album was released in 1976?,"""{\""text\"": [\""A Day at the Races\""], \""answer..."
2,"In the 1874 general election, Disraeli was ret...",What removed Catholic Rituals from Anglican se...,"""{\""text\"": [\""Public Worship Regulation Act 1..."
3,"Imperial College Union, the students' union at...",How long is the tenure for an officer to run t...,"""{\""text\"": [\""one year\""], \""answer_start\"": ..."
4,"In 2010, 24.9 percent of households reported h...",Percentage of unwed births?,"""{\""text\"": [\""56\""], \""answer_start\"": [635]}"""
...,...,...,...
495,Coyotes and big cats have also been known to a...,What big cats in Indonesia also attack dogs?,"""{\""text\"": [\""Tigers\""], \""answer_start\"": [2..."
496,Spectre opened in Germany with $22.45 million ...,How much more did Spectre earn compared with S...,"""{\""text\"": [\""4%\""], \""answer_start\"": [849]}"""
497,Portland is known for being a trailblazer in v...,How many $10+ donations must Portland city cou...,"""{\""text\"": [\""200\""], \""answer_start\"": [257]}"""
498,The first boardwalk was built in 1870 along a ...,Why was the first boardwalk built in New Orleans?,"""{\""text\"": [\""to help hotel owners keep sand ..."


# Temporal

In [41]:
t = crossproduct(editor.template(
    {
        'contexts': [
            'Both {first_name} and {first_name2} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
            'Both {first_name2} and {first_name} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
        ],
        'qas': [
            (
                'Who is {a:prof2}?',
                '{first_name}'
            ),
        ]

    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'There was a change in profession'
# test = MFT(**t, expect=expect_squad, capability='Temporal', name=name, description='' )
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "temproal")

,context,question,answers
0,"Both Suzanne and Victoria were administrators,...",Who is an assistant?,"{""text"": [""Suzanne""], ""answer_start"": [5]}"
1,"Both Victoria and Suzanne were administrators,...",Who is an assistant?,"{""text"": [""Suzanne""], ""answer_start"": [18]}"
2,"Both Edith and James were educators, but there...",Who is an agent?,"{""text"": [""Edith""], ""answer_start"": [5]}"
3,"Both James and Edith were educators, but there...",Who is an agent?,"{""text"": [""Edith""], ""answer_start"": [15]}"
4,"Both David and Katie were models, but there wa...",Who is an investigator?,"{""text"": [""David""], ""answer_start"": [5]}"
...,...,...,...
965,"Both Margaret and Howard were assistants, but ...",Who is a journalist?,"{""text"": [""Howard""], ""answer_start"": [18]}"
966,"Both Lawrence and Leslie were investors, but t...",Who is an editor?,"{""text"": [""Lawrence""], ""answer_start"": [5]}"
967,"Both Leslie and Lawrence were investors, but t...",Who is an editor?,"{""text"": [""Lawrence""], ""answer_start"": [16]}"
968,"Both Lawrence and Jean were photographers, but...",Who is an administrator?,"{""text"": [""Lawrence""], ""answer_start"": [5]}"


In [42]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} became a {prof} before {first_name2} did.',
            '{first_name2} became a {prof} after {first_name} did.',
        ],
        'qas': [
            (
                'Who became a {prof} first?',
                '{first_name}'
            ),
            (
                'Who became a {prof} last?',
                '{first_name2}'
            ),
        ]

    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Understanding before / after -> first / last.'
# test = MFT(**t, expect=expect_squad, capability='Temporal', name=name, description='' )
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "before_after")

,context,question,answers
0,Nicole became a interpreter before Andrew did.,Who became a interpreter first?,"{""text"": [""Nicole""], ""answer_start"": [0]}"
1,Nicole became a interpreter before Andrew did.,Who became a interpreter last?,"{""text"": [""Andrew""], ""answer_start"": [35]}"
2,Andrew became a interpreter after Nicole did.,Who became a interpreter first?,"{""text"": [""Nicole""], ""answer_start"": [34]}"
3,Andrew became a interpreter after Nicole did.,Who became a interpreter last?,"{""text"": [""Andrew""], ""answer_start"": [0]}"
4,Jerry became a photographer before Albert did.,Who became a photographer first?,"{""text"": [""Jerry""], ""answer_start"": [0]}"
...,...,...,...
1991,Patrick became a attorney after Mark did.,Who became a attorney last?,"{""text"": [""Patrick""], ""answer_start"": [0]}"
1992,Robert became a investor before Alex did.,Who became a investor first?,"{""text"": [""Robert""], ""answer_start"": [0]}"
1993,Robert became a investor before Alex did.,Who became a investor last?,"{""text"": [""Alex""], ""answer_start"": [32]}"
1994,Alex became a investor after Robert did.,Who became a investor first?,"{""text"": [""Robert""], ""answer_start"": [29]}"


# Negation

In [43]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is not {a:prof}. {first_name2} is.',
            '{first_name2} is {a:prof}. {first_name} is not.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name2}'
            ),
            (
                'Who is not {a:prof}?',
                '{first_name}'
            ),
        ]

    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Negation in context, may or may not be in question'
# test = MFT(**t, expect=expect_squad, capability='Negation', name=name, description='' )
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "negation")

,context,question,answers
0,Tim is not an educator. Roger is.,Who is an educator?,"{""text"": [""Roger""], ""answer_start"": [24]}"
1,Tim is not an educator. Roger is.,Who is not an educator?,"{""text"": [""Tim""], ""answer_start"": [0]}"
2,Roger is an educator. Tim is not.,Who is an educator?,"{""text"": [""Roger""], ""answer_start"": [0]}"
3,Roger is an educator. Tim is not.,Who is not an educator?,"{""text"": [""Tim""], ""answer_start"": [22]}"
4,Heather is not a secretary. George is.,Who is a secretary?,"{""text"": [""George""], ""answer_start"": [28]}"
...,...,...,...
1975,Charles is an educator. Brian is not.,Who is not an educator?,"{""text"": [""Brian""], ""answer_start"": [24]}"
1976,Dorothy is not an interpreter. Amanda is.,Who is an interpreter?,"{""text"": [""Amanda""], ""answer_start"": [31]}"
1977,Dorothy is not an interpreter. Amanda is.,Who is not an interpreter?,"{""text"": [""Dorothy""], ""answer_start"": [0]}"
1978,Amanda is an interpreter. Dorothy is not.,Who is an interpreter?,"{""text"": [""Amanda""], ""answer_start"": [0]}"


In [44]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:prof}. {first_name2} is {a:prof2}.',
            '{first_name2} is {a:prof2}. {first_name} is {a:prof}.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name}'
            ),
            (
                'Who is not {a:prof}?',
                '{first_name2}'
            ),
            (
                'Who is {a:prof2}?',
                '{first_name2}'
            ),
            (
                'Who is not {a:prof2}?',
                '{first_name}'
            ),
        ]

    },
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Negation in question only.'
# test = MFT(**t, expect=expect_squad, capability='Negation', name=name, description='' )
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "negation_question")

,context,question,answers
0,Michael is an educator. John is an activist.,Who is an educator?,"{""text"": [""Michael""], ""answer_start"": [0]}"
1,Michael is an educator. John is an activist.,Who is not an educator?,"{""text"": [""John""], ""answer_start"": [24]}"
2,Michael is an educator. John is an activist.,Who is an activist?,"{""text"": [""John""], ""answer_start"": [24]}"
3,Michael is an educator. John is an activist.,Who is not an activist?,"{""text"": [""Michael""], ""answer_start"": [0]}"
4,John is an activist. Michael is an educator.,Who is an educator?,"{""text"": [""Michael""], ""answer_start"": [21]}"
...,...,...,...
3835,Lisa is a reporter. Stephen is an author.,Who is not an author?,"{""text"": [""Lisa""], ""answer_start"": [0]}"
3836,Stephen is an author. Lisa is a reporter.,Who is a reporter?,"{""text"": [""Lisa""], ""answer_start"": [22]}"
3837,Stephen is an author. Lisa is a reporter.,Who is not a reporter?,"{""text"": [""Stephen""], ""answer_start"": [0]}"
3838,Stephen is an author. Lisa is a reporter.,Who is an author?,"{""text"": [""Stephen""], ""answer_start"": [0]}"


# Fairness

In [45]:
import collections
fewer_profs = ['doctor', 'nurse', 'secretary', 'CEO']
t = editor.template(
    [
        ('{male} is not {a:prof}, {female} is.', 'Who is {a:prof}?', '{female}', 'woman', '{prof}'),
        ('{female} is not {a:prof}, {male} is.', 'Who is {a:prof}?', '{male}', 'man', '{prof}'),
    ],
#     prof=professions + ['doctor'],
    prof=fewer_profs,
    remove_duplicates=True,
    nsamples=1000,
    unroll=True,
    save=True,
    )
data = [(d[0], d[1]) for d in t.data]
labels = [d[2] for d in t.data]
meta = [(d[3], d[4]) for d in t.data]

# test = MFT(data, expect=expect_squad, labels=labels, meta=meta, templates=t.templates,
#           name='M/F failure rates should be similar for different professions', capability='Fairness',
#           description='Using negation in context.')
# test.run(predconfs, n=100)

# suite.add(test)
t.data = [[i] for i in data]
t.labels = [[i] for i in labels]
dataset_fmt(t, "fair")



,context,question,answers
0,"Greg is not a doctor, Helen is.",Who is a doctor?,"{""text"": [""Helen""], ""answer_start"": [22]}"
1,"Helen is not a doctor, Greg is.",Who is a doctor?,"{""text"": [""Greg""], ""answer_start"": [23]}"
2,"Donald is not a doctor, Barbara is.",Who is a doctor?,"{""text"": [""Barbara""], ""answer_start"": [24]}"
3,"Barbara is not a doctor, Donald is.",Who is a doctor?,"{""text"": [""Donald""], ""answer_start"": [25]}"
4,"Joseph is not a nurse, Elizabeth is.",Who is a nurse?,"{""text"": [""Elizabeth""], ""answer_start"": [23]}"
...,...,...,...
1975,"Amy is not a CEO, Scott is.",Who is a CEO?,"{""text"": [""Scott""], ""answer_start"": [18]}"
1976,"Kevin is not a nurse, Sharon is.",Who is a nurse?,"{""text"": [""Sharon""], ""answer_start"": [22]}"
1977,"Sharon is not a nurse, Kevin is.",Who is a nurse?,"{""text"": [""Kevin""], ""answer_start"": [23]}"
1978,"Louis is not a nurse, Evelyn is.",Who is a nurse?,"{""text"": [""Evelyn""], ""answer_start"": [22]}"


# Coref

In [46]:
if 'actress' in professions:
    professions.remove('actress')
t = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{female} and {male} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{male} and {female} are friends. She is {a:prof2}, and he is {a:prof1}.',
            '{female} and {male} are friends. She is {a:prof2}, and he is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{male}'
            ),
            (
                'Who is {a:prof2}?',
                '{female}'
            ),
        ]

    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Basic coref, he / she'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "basic_coref")

,context,question,answers
0,"Bobby and Louise are friends. He is an author,...",Who is an author?,"{""text"": [""Bobby""], ""answer_start"": [0]}"
1,"Bobby and Louise are friends. He is an author,...",Who is an administrator?,"{""text"": [""Louise""], ""answer_start"": [10]}"
2,"Louise and Bobby are friends. He is an author,...",Who is an author?,"{""text"": [""Bobby""], ""answer_start"": [11]}"
3,"Louise and Bobby are friends. He is an author,...",Who is an administrator?,"{""text"": [""Louise""], ""answer_start"": [0]}"
4,Bobby and Louise are friends. She is an admini...,Who is an author?,"{""text"": [""Bobby""], ""answer_start"": [0]}"
...,...,...,...
3891,"Eleanor and Roger are friends. He is a nurse, ...",Who is an economist?,"{""text"": [""Eleanor""], ""answer_start"": [0]}"
3892,Roger and Eleanor are friends. She is an econo...,Who is a nurse?,"{""text"": [""Roger""], ""answer_start"": [0]}"
3893,Roger and Eleanor are friends. She is an econo...,Who is an economist?,"{""text"": [""Eleanor""], ""answer_start"": [10]}"
3894,Eleanor and Roger are friends. She is an econo...,Who is a nurse?,"{""text"": [""Roger""], ""answer_start"": [12]}"


In [47]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. His mom is {a:prof}.',
            '{female} and {male} are friends. His mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{male}'
            ),
        ]

    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=250,
    ))
t += crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. Her mom is {a:prof}.',
            '{female} and {male} are friends. Her mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{female}'
            ),
        ]

    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=250,
    ))

name = 'Basic coref, his / her'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "basic_coref2")

,context,question,answers
0,Jay and Eleanor are friends. His mom is an act...,Whose mom is an activist?,"{""text"": [""Jay""], ""answer_start"": [0]}"
1,Eleanor and Jay are friends. His mom is an act...,Whose mom is an activist?,"{""text"": [""Jay""], ""answer_start"": [12]}"
2,Hugh and Katie are friends. His mom is an arch...,Whose mom is an architect?,"{""text"": [""Hugh""], ""answer_start"": [0]}"
3,Katie and Hugh are friends. His mom is an arch...,Whose mom is an architect?,"{""text"": [""Hugh""], ""answer_start"": [10]}"
4,Tom and Rachel are friends. His mom is a journ...,Whose mom is a journalist?,"{""text"": [""Tom""], ""answer_start"": [0]}"
...,...,...,...
995,Sally and Kevin are friends. Her mom is an arc...,Whose mom is an architect?,"{""text"": [""Sally""], ""answer_start"": [0]}"
996,Howard and Marilyn are friends. Her mom is a p...,Whose mom is a producer?,"{""text"": [""Marilyn""], ""answer_start"": [11]}"
997,Marilyn and Howard are friends. Her mom is a p...,Whose mom is a producer?,"{""text"": [""Marilyn""], ""answer_start"": [0]}"
998,Matt and Lynn are friends. Her mom is an inves...,Whose mom is an investor?,"{""text"": [""Lynn""], ""answer_start"": [9]}"


In [48]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} and {first_name2} are friends. The former is {a:prof1}.',
            '{first_name2} and {first_name} are friends. The latter is {a:prof1}.',
            '{first_name} and {first_name2} are friends. The former is {a:prof1} and the latter is {a:prof2}.',
            '{first_name2} and {first_name} are friends. The former is {a:prof2} and the latter is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{first_name}'
            ),
        ]

    },
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Former / Latter'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "former")

,context,question,answers
0,Julia and Rose are friends. The former is an e...,Who is an entrepreneur?,"{""text"": [""Julia""], ""answer_start"": [0]}"
1,Rose and Julia are friends. The latter is an e...,Who is an entrepreneur?,"{""text"": [""Julia""], ""answer_start"": [9]}"
2,Julia and Rose are friends. The former is an e...,Who is an entrepreneur?,"{""text"": [""Julia""], ""answer_start"": [0]}"
3,Rose and Julia are friends. The former is a jo...,Who is an entrepreneur?,"{""text"": [""Julia""], ""answer_start"": [9]}"
4,Johnny and Susan are friends. The former is an...,Who is an engineer?,"{""text"": [""Johnny""], ""answer_start"": [0]}"
...,...,...,...
1951,Donald and Judy are friends. The former is a w...,Who is an educator?,"{""text"": [""Judy""], ""answer_start"": [11]}"
1952,Patricia and Jason are friends. The former is ...,Who is an assistant?,"{""text"": [""Patricia""], ""answer_start"": [0]}"
1953,Jason and Patricia are friends. The latter is ...,Who is an assistant?,"{""text"": [""Patricia""], ""answer_start"": [10]}"
1954,Patricia and Jason are friends. The former is ...,Who is an assistant?,"{""text"": [""Patricia""], ""answer_start"": [0]}"


# SRL

In [49]:
import pattern
import pattern.en
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
#a = pattern.en.tenses('loves')[0]
#b = pattern.en.tenses('stolen')[0]
#pverb = [(pattern.en.conjugate(v, *a), pattern.en.conjugate(v, *b)) for v in pverb]

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} {v[0]} {first_name2}.',
            '{first_name2} is {v[1]} by {first_name}.',
        ],
        'qas': [
            (
                'Who {v[0]}?',
                '{first_name}'
            ),
            (
                'Who is {v[1]}?',
                '{first_name2}'
            ),
        ]

    },
    v=pverb,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Agent / object distinction'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='SRL')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "agent")

,context,question,answers
0,Kathleen h Don.,Who h?,"{""text"": [""Kathleen""], ""answer_start"": [0]}"
1,Kathleen h Don.,Who is a?,"{""text"": [""Don""], ""answer_start"": [11]}"
2,Don is a by Kathleen.,Who h?,"{""text"": [""Kathleen""], ""answer_start"": [12]}"
3,Don is a by Kathleen.,Who is a?,"{""text"": [""Don""], ""answer_start"": [0]}"
4,Virginia a Nancy.,Who a?,"{""text"": [""Virginia""], ""answer_start"": [0]}"
...,...,...,...
1977,Carl is o by William.,Who is o?,"{""text"": [""Carl""], ""answer_start"": [0]}"
1978,Andrew b Patricia.,Who b?,"{""text"": [""Andrew""], ""answer_start"": [0]}"
1979,Andrew b Patricia.,Who is l?,"{""text"": [""Patricia""], ""answer_start"": [9]}"
1980,Patricia is l by Andrew.,Who b?,"{""text"": [""Andrew""], ""answer_start"": [17]}"


In [50]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} {v[0]} {first_name2}. {first_name2} {v[0]} {first_name3}.',
            '{first_name} {v[0]} {first_name2}. {first_name3} is {v[1]} by {first_name2}.',
            '{first_name2} is {v[1]} by {first_name}. {first_name2} {v[0]} {first_name3}.',
            '{first_name2} is {v[1]} by {first_name}. {first_name3} is {v[1]} by {first_name2}.',
        ],
        'qas': [
            (
                'Who {v[0]} {first_name2}?',
                '{first_name}'
            ),
            (
                'Who {v[0]} {first_name3}?',
                '{first_name2}'
            ),
            (
                'Who is {v[1]} by {first_name}?',
                '{first_name2}'
            ),
            (
                'Who is {v[1]} by {first_name2}?',
                '{first_name3}'
            ),
        ]

    },
    save=True,
    v=pverb,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Agent / object distinction with 3 agents'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='SRL')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)
dataset_fmt(t, "agent2")

,context,question,answers
0,Alexandra r Jane. Jane r Lawrence.,Who r Jane?,"{""text"": [""Alexandra""], ""answer_start"": [0]}"
1,Alexandra r Jane. Jane r Lawrence.,Who r Lawrence?,"{""text"": [""Jane""], ""answer_start"": [12]}"
2,Alexandra r Jane. Jane r Lawrence.,Who is e by Alexandra?,"{""text"": [""Jane""], ""answer_start"": [12]}"
3,Alexandra r Jane. Jane r Lawrence.,Who is e by Jane?,"{""text"": [""Lawrence""], ""answer_start"": [25]}"
4,Alexandra r Jane. Lawrence is e by Jane.,Who r Jane?,"{""text"": [""Alexandra""], ""answer_start"": [0]}"
...,...,...,...
7867,Andrea is e by Florence. Andrea r Billy.,Who is e by Andrea?,"{""text"": [""Billy""], ""answer_start"": [34]}"
7868,Andrea is e by Florence. Billy is e by Andrea.,Who r Andrea?,"{""text"": [""Florence""], ""answer_start"": [15]}"
7869,Andrea is e by Florence. Billy is e by Andrea.,Who r Billy?,"{""text"": [""Andrea""], ""answer_start"": [0]}"
7870,Andrea is e by Florence. Billy is e by Andrea.,Who is e by Florence?,"{""text"": [""Andrea""], ""answer_start"": [0]}"


# Combine all

In [52]:
import pandas as pd
import glob

path = 'new_data'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
del li
frame

,context,question,answers
0,Martha has a dog and a SUV.,What animal does Martha have?,"{""text"": [""dog""], ""answer_start"": [13]}"
1,Martha has a dog and a SUV.,What vehicle does Martha have?,"{""text"": [""SUV""], ""answer_start"": [23]}"
2,Martha has a SUV and a dog.,What animal does Martha have?,"{""text"": [""dog""], ""answer_start"": [23]}"
3,Martha has a SUV and a dog.,What vehicle does Martha have?,"{""text"": [""SUV""], ""answer_start"": [13]}"
4,Stephanie has a snake and a train.,What animal does Stephanie have?,"{""text"": [""snake""], ""answer_start"": [16]}"
...,...,...,...
59318,"Amy is not a CEO, Scott is.",Who is a CEO?,"{""text"": [""Scott""], ""answer_start"": [18]}"
59319,"Kevin is not a nurse, Sharon is.",Who is a nurse?,"{""text"": [""Sharon""], ""answer_start"": [22]}"
59320,"Sharon is not a nurse, Kevin is.",Who is a nurse?,"{""text"": [""Kevin""], ""answer_start"": [23]}"
59321,"Louis is not a nurse, Evelyn is.",Who is a nurse?,"{""text"": [""Evelyn""], ""answer_start"": [22]}"


In [53]:
frame.iloc[0].answers

'{"text": ["dog"], "answer_start": [13]}'

In [54]:
json.loads(frame.iloc[57410].answers)

{'text': ['Bill'], 'answer_start': [26]}

In [55]:
{'text': ['primary (P), secondary (S), and tertiary (T)'],
 'answer_start': [115]}
frame.to_csv('checklist_train_1130.csv', index=None)
combined = pd.concat([
    df_train[['context','question','answers']],
    frame]
)
## shuffle
combined.iloc[np.random.permutation(len(combined))].reset_index(drop=True)

,context,question,answers
0,"The inflection of determinatives is complex, s...",What language does Catalan have more contracti...,"{""text"": [""Spanish""], ""answer_start"": [200]}"
1,Harry is warmer than Jerry.,Who is warmer?,"{""text"": [""Harry""], ""answer_start"": [0]}"
2,The mandolin's popularity in the United States...,What group in the US was popular?,"{""text"": [""Estudiantina Figaro""], ""answer_star..."
3,"The Clerk of the Parliaments, an official of t...",What phrase is used to announce a withheld ass...,"{""text"": [""\""La Reyne s'avisera\""""], ""answer_s..."
4,Deborah is less polite than Kathleen.,Who is less rude?,"{""text"": [""Kathleen""], ""answer_start"": [28]}"
...,...,...,...
176917,City transportation in Strasbourg includes the...,What is the length of the Strasbourg tramway i...,"{""text"": [""55.8 km""], ""answer_start"": [243]}"
176918,The Arctic tern holds the long-distance migrat...,Where do Manx shearwaters migrate?,"{""text"": [""between their northern breeding gro..."
176919,The foundations of the British Empire were lai...,When was John Cabot's voyage commissioned?,"{""text"": [""1496""], ""answer_start"": [101]}"
176920,"Uniquely among major American cities, New York...",What square is home to the US Court of Interna...,"{""text"": [""Foley Square""], ""answer_start"": [224]}"


In [56]:

merged_df = pd.concat([combined, df_train_selected], axis=0)

# Reset the index if necessary
merged_df = merged_df.reset_index(drop=True)

In [57]:
merged_df.to_csv('combined_train_1130.csv', index=None)

# Other

In [59]:
#將檔案copy回去
!cp -r /content/final/checklist_train_1130.csv /content/MyDrive/MyDrive/UT_NLP/final/checklist_train_1130.csv

!cp -r /content/final/combined_train_1130.csv /content/MyDrive/MyDrive/UT_NLP/final/combined_train_1130.csv


!cp -r /content/final/train_merge.csv /content/MyDrive/MyDrive/UT_NLP/final/train_merge.csv

!cp -r /content/final/new_data_1130 /content/MyDrive/MyDrive/UT_NLP/final/

In [ ]:
#將檔案copy回去
#!cp -r /content/final/trained_model.zip /content/MyDrive/MyDrive/UT_NLP/final/trained_model.zip
!cp -r /content/final/run.py /content/MyDrive/MyDrive/UT_NLP/final/run.py

#!cp -r /content/a4-distrib/factchecking_main.py /content/MyDrive/MyDrive/UT_NLP/a4-distrib/factchecking_main.py
#!cp -r /content/a4-distrib/transformer.py /content/MyDrive/MyDrive/UT_NLP/a4-distrib/transformer.py

In [ ]:
!cp -r /content/final /content/MyDrive/MyDrive/UT_NLP


# 新增區段

In [ ]:
#copy zip file
!cp -r /content/a4-distrib/wc22987.zip /content/MyDrive/MyDrive/UT_NLP/a4-distrib/